In [1]:
import json


# 读取数据
with open("../xfdata/dataset/job_list.json", "r", encoding="utf-8") as f:
    job_list = json.load(f)
with open("../xfdata/dataset/train.json", "r", encoding="utf-8") as f:
    train = json.load(f)
with open("../xfdata/dataset/test.json", "r", encoding="utf-8") as f:
    test = json.load(f)
    
# 1、教育经历
with open("../user_data/profileEduExps_sentences.json", "r", encoding="utf-8") as f:
    profileEduExps_sentences = json.load(f)

# 2、社会经历
with open("../user_data/profileSocialExps_sentences.json", "r", encoding="utf-8") as f:
    profileSocialExps_sentences = json.load(f)

# 3、项目经历
with open("../user_data/profileProjectExps_sentences.json", "r", encoding="utf-8") as f:
    profileProjectExps_sentences = json.load(f)

# 4、工作经历
with open("../user_data/profileWorkExps_sentences.json", "r", encoding="utf-8") as f:
    profileWorkExps_sentences = json.load(f)

# 5、技能
with open("../user_data/profileSkills_sentences.json", "r", encoding="utf-8") as f:
    profileSkills_sentences = json.load(f)

# 6、荣誉
with open("../user_data/profileAwards_sentences.json", "r", encoding="utf-8") as f:
    profileAwards_sentences = json.load(f)

# 7、求职意愿
with open("../user_data/profileDesire_sentences.json", "r", encoding="utf-8") as f:
    profileDesire_sentences = json.load(f)

# 8、语言
with open("../user_data/profileLanguage_sentences.json", "r", encoding="utf-8") as f:
    profileLanguage_sentences = json.load(f)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import torch


# 获取标签
labels = [i["positionID"] for i in train]
labelencoder = LabelEncoder()
labels = labelencoder.fit_transform(labels)
print(len(np.unique(labels)))

51


# 使用BERT进行预测

In [3]:
from sklearn.model_selection import train_test_split



sentences = []
# 调整位置
for s1, s2, s3, s4, s5, s6, s7, s8 in zip(profileEduExps_sentences, profileWorkExps_sentences, profileProjectExps_sentences, \
profileSocialExps_sentences, profileSkills_sentences, profileAwards_sentences, profileDesire_sentences, \
profileLanguage_sentences):
    sentences.append(s1 + s2 + s3 + s4 + s5 + s6 + s7 + s8)

train_sentences, test_sentences = sentences[: len(train)], sentences[len(train): ]
print(len(train_sentences), len(test_sentences))
train_idx = np.load("../user_data/train_idx.npy")
valid_idx = np.load("../user_data/valid_idx.npy")
train_sentences, valid_sentences = [train_sentences[idx] for idx in train_idx], [train_sentences[idx] for idx in valid_idx]
train_labels, valid_labels = [labels[idx] for idx in train_idx], [labels[idx] for idx in valid_idx]
print(len(train_sentences), len(train_labels))
print(len(valid_sentences), len(valid_labels))

20000 6500
16000 16000
4000 4000


In [4]:
from datasets import Dataset


train_dataset = Dataset.from_dict({
    "text": train_sentences,
    "label": train_labels
})
valid_dataset = Dataset.from_dict({
    "text": valid_sentences,
    "label": valid_labels
})
test_dataset = Dataset.from_dict({
    "text": test_sentences
})

In [6]:
from transformers import BertForSequenceClassification, BertTokenizer


path = "../user_data/classification_output_0/checkpoint-1500"
model = BertForSequenceClassification.from_pretrained(path)
tokenizer = BertTokenizer.from_pretrained(path)

In [7]:
def preprocess(examples):
    """单个处理"""
    input_ids, token_type_ids, attention_mask = tokenizer.encode_plus(
        examples["text"],
        padding="max_length",
        max_length=1024,
        truncation=True,
        return_tensors="pt"
    ).values()
    examples["input_ids"] = input_ids[0]
    examples["token_type_ids"] = token_type_ids[0]
    examples["attention_mask"] = attention_mask[0]
    return examples


train_encoded = train_dataset.map(preprocess)
valid_encoded = valid_dataset.map(preprocess)
test_encoded = test_dataset.map(preprocess)

train_encoded.set_format("torch", columns=["label", "input_ids", "token_type_ids", "attention_mask"])
valid_encoded.set_format("torch", columns=["label", "input_ids", "token_type_ids", "attention_mask"])
test_encoded.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask"])

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)
def get_logits(batch):
    with torch.no_grad():
        input_ids, token_type_ids, attention_mask = batch["input_ids"].to(device), batch["token_type_ids"].to(device), batch["attention_mask"].to(device)
        result = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        batch["logits"] = result.logits
        return batch

train_encoded = train_encoded.map(get_logits, batched=True, batch_size=64)
valid_encoded = valid_encoded.map(get_logits, batched=True, batch_size=64)
test_encoded = test_encoded.map(get_logits, batched=True, batch_size=64)

train_encoded.set_format("torch", columns=["label", "input_ids", "token_type_ids", "attention_mask", "logits"])
valid_encoded.set_format("torch", columns=["label", "input_ids", "token_type_ids", "attention_mask", "logits"])
test_encoded.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask", "logits"])
print("done!")

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

done!


In [10]:
from datasets import DatasetDict


dataset_for_nsp = DatasetDict({
    "train": train_encoded,
    "valid": valid_encoded,
    "test": test_encoded
})
dataset_for_nsp.save_to_disk("../user_data/dataset_for_nsp")

Saving the dataset (0/1 shards):   0%|          | 0/16000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6500 [00:00<?, ? examples/s]